In [1]:
import pandas as pd

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

/var/folders/hk/p7fcnhm10z92trzs_9kwqlyr0000gn/T/ipykernel_23277/881960093.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('OriginalFiles/births_train.csv')

In [3]:
data.head()

,INFANT_ALIVE_AT_REPORT,BIRTH_YEAR,BIRTH_MONTH,BIRTH_PLACE,MOTHER_AGE_YEARS,MOTHER_RACE_6CODE,MOTHER_EDUCATION,FATHER_COMBINED_AGE,FATHER_EDUCATION,MONTH_PRECARE_RECODE,CIG_BEFORE,CIG_1_TRI,CIG_2_TRI,CIG_3_TRI,MOTHER_HEIGHT_IN,MOTHER_BMI_RECODE,MOTHER_PRE_WEIGHT,MOTHER_DELIVERY_WEIGHT,MOTHER_WEIGHT_GAIN,DIABETES_PRE,DIABETES_GEST,HYP_TENS_PRE,HYP_TENS_GEST,PREV_BIRTH_PRETERM,NO_RISK,NO_INFECTIONS_REPORTED,LABOR_IND,LABOR_AUGM,STEROIDS,ANTIBIOTICS,ANESTHESIA,DELIV_METHOD_RECODE_COMB,ATTENDANT_BIRTH,APGAR_5,APGAR_5_RECODE,APGAR_10,APGAR_10_RECODE,INFANT_SEX,OBSTETRIC_GESTATION_WEEKS,INFANT_WEIGHT_GRAMS,INFANT_ASSIST_VENTI,INFANT_ASSIST_VENTI_6HRS,INFANT_NICU_ADMISSION,INFANT_SURFACANT,INFANT_ANTIBIOTICS,INFANT_SEIZURES,INFANT_NO_ABNORMALITIES,INFANT_ANCEPHALY,INFANT_MENINGOMYELOCELE,INFANT_LIMB_REDUCTION,INFANT_DOWN_SYNDROME,INFANT_SUSPECTED_CHROMOSOMAL_DISORDER,INFANT_NO_CONGENITAL_ANOMALIES_CHECKED,INFANT_BREASTFED
0,N,2015,2,1,29,3,9,99,9,4,99,99,99,99,99,9,999,999,99,N,N,N,N,N,1,1,N,N,N,Y,N,2,1,4,2,3,1,F,35,2770,N,N,Y,N,N,N,0,N,N,N,N,N,0,N
1,N,2015,2,1,22,1,3,29,4,1,0,0,0,0,65,4,180,198,18,N,N,N,N,N,1,1,N,N,N,N,N,2,1,5,2,7,3,F,35,1191,N,N,Y,N,N,N,0,N,N,N,N,C,0,N
2,N,2015,2,1,38,1,4,40,3,1,0,0,0,0,63,3,155,167,12,N,N,N,N,N,1,1,N,N,N,Y,N,1,1,1,1,1,1,F,18,9999,U,U,U,U,U,U,9,N,N,N,N,N,1,N
3,N,2015,4,1,39,2,7,42,6,1,0,0,0,0,60,3,128,152,24,N,N,N,N,Y,0,1,N,N,N,Y,Y,1,1,8,3,88,5,F,37,1925,N,N,N,N,N,Y,0,Y,N,N,N,N,0,N
4,N,2015,4,1,18,3,2,99,9,2,6,4,2,2,61,2,110,130,20,N,N,N,N,N,1,1,N,N,N,N,N,1,1,3,1,3,1,M,32,670,U,U,U,U,U,U,9,N,N,N,N,N,1,N


In [5]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
# 初始化spark会话
spark = SparkSession.builder.appName('birthrate').getOrCreate()

24/02/22 11:59:19 WARN Utils: Your hostname, dongyanshens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.116 instead (on interface en0)
24/02/22 11:59:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 11:59:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# pandas转spark
values = data.values.tolist()
columns = data.columns.tolist()
spark_df = spark.createDataFrame(values, columns)

In [7]:
spark_df.show(10)

24/02/22 12:00:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------------+----------+-----------+-----------+----------------+-----------------+----------------+-------------------+----------------+--------------------+----------+---------+---------+---------+----------------+-----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+-------+----------------------+---------+----------+--------+-----------+----------+------------------------+---------------+-------+--------------+--------+---------------+----------+-------------------------+-------------------+-------------------+------------------------+---------------------+----------------+------------------+---------------+-----------------------+----------------+-----------------------+---------------------+--------------------+-------------------------------------+--------------------------------------+----------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_YEAR|BIRTH_MONTH|BIRTH_PLACE|MOTHER_AGE_

In [12]:
spark_df \
        .repartition(2) \
        .write.mode('overwrite').format("csv") \
        .option("encoding","utf-8").option("header",True) \
        .save("/Users/dongyanshen/Desktop/DYSProjects/TBProject/BirthRate_24F19/BirthRate_24F19/births_train.csv")

In [13]:
df = spark.read.format("csv") \
        .load('/Users/dongyanshen/Desktop/DYSProjects/TBProject/BirthRate_24F19/BirthRate_24F19/births_train.csv', 
              header=True, inferSchema=True, encoding="utf-8", sep=',')

In [14]:
df.show(10)

+----------------------+----------+-----------+-----------+----------------+-----------------+----------------+-------------------+----------------+--------------------+----------+---------+---------+---------+----------------+-----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+-------+----------------------+---------+----------+--------+-----------+----------+------------------------+---------------+-------+--------------+--------+---------------+----------+-------------------------+-------------------+-------------------+------------------------+---------------------+----------------+------------------+---------------+-----------------------+----------------+-----------------------+---------------------+--------------------+-------------------------------------+--------------------------------------+----------------+
|INFANT_ALIVE_AT_REPORT|BIRTH_YEAR|BIRTH_MONTH|BIRTH_PLACE|MOTHER_AGE_

In [16]:
train_data, val_data, test_data = df.randomSplit([0.5, 0.2, 0.3], seed=42)

In [20]:
train_data.count(), val_data.count(), test_data.count()

(22824, 9154, 13451)

In [ ]:
from pyspark.ml.feature import VectorAssembler

inputCols = df.columns[1:]
outputCol = df.columns[0]

Assembler = VectorAssembler(inputCols=inputCols, outputCol='features')

train_data = Assembler.transform(train_data)
train_data = Assembler.transform(val_data)
test_data = Assembler.transform(test_data)

In [ ]:
!dfs -ls

zsh:1: command not found: dfs


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression()